<p style ="text-align:center">
    <img src="http://epecora.com.br/DataFiles/BannerUFPR.png" width="700" alt="PPGOLD/PPGMNE Python:INTRO"  />
</p>


# SAOP09 - Data Science For Business
## Prof. Eduardo Pécora

# **Gini Index**
Tempo estimado: **30** minutos

## Objetivos

Após completar esta aula, você será capaz de:

* Entender como é calculado o Gini Index
* Usar o Gini na separação de conjuntos


### Dados

Esse código usa o DataFrame fictício sobre a escolha de cervejas, com os atributos:

* IBU (amargor),
* Teor_Alcool (em %),
* Cor_SRM (que é depois categorizada em Clara ou Escura), e
* Aceita (classe: “Sim” ou “Não”).

In [98]:
import pandas as pd

# Dados exemplo sobre cervejas
data = {
    'IBU': [15, 60, 20, 35, 35, 80, 10, 70, 25, 35],
    'Teor_Alcool': [6.2, 4.5, 4.8, 7.0, 5.5, 8.0, 4.0, 6.2, 5.0, 6.8],
    'Cor': ["Clara", "Clara", "Clara", "Escura", "Clara", "Escura", "Clara", "Escura", "Clara", "Escura"],
    'Aceita': ['Sim', 'Não', 'Sim', 'Não', 'Não', 'Não', 'Sim', 'Sim', 'Sim', 'Não']
}

df = pd.DataFrame(data)

df

,IBU,Teor_Alcool,Cor,Aceita
0,15,6.2,Clara,Sim
1,60,4.5,Clara,Não
2,20,4.8,Clara,Sim
3,35,7.0,Escura,Não
4,35,5.5,Clara,Não
5,80,8.0,Escura,Não
6,10,4.0,Clara,Sim
7,70,6.2,Escura,Sim
8,25,5.0,Clara,Sim
9,35,6.8,Escura,Não


## O que é o Gini Index?

O Gini Index mede a "impureza" de um conjunto de dados, ou como eu digo o quão bom (puros) os dados ficarão ao separarmos. Quanto menor o índice menor a impureza.

Para um problema com duas classes temos:

$Gini = 1 - p_1^2 - p_2^2$

Para um problema com várias classes temos:

$Gini = 1 - \sum_{c \in Classes} p_c^2 $


# Exemplo

## Calcular o Gini para a Cor

### 1. Separar o DF original em dois DFs (grupos - claro e escuro)

In [99]:
group_clara = df[df['Cor'] == 'Clara']
group_escura = df[df['Cor'] == 'Escura']

print(group_clara)
print(group_escura)

   IBU  Teor_Alcool    Cor Aceita
0   15          6.2  Clara    Sim
1   60          4.5  Clara    Não
2   20          4.8  Clara    Sim
4   35          5.5  Clara    Não
6   10          4.0  Clara    Sim
8   25          5.0  Clara    Sim
   IBU  Teor_Alcool     Cor Aceita
3   35          7.0  Escura    Não
5   80          8.0  Escura    Não
7   70          6.2  Escura    Sim
9   35          6.8  Escura    Não


## 2. Calcula o Gini para cada classe

### Clara

In [100]:
size_clara = len(group_clara['Aceita'])
print(f"Tamanho do DF Clara: {size_clara}\n")

gr = group_clara.groupby("Aceita")["Cor"].count()
print(f"Quantidade de elementos em cada classe:")
print(gr)

print(f"\n% de elementos em cada classe:")
gr = gr/size_clara
print(gr)

print(f"\n% de elementos ao quadrado em cada classe:")
gr = gr**2
print(gr)

print(f"\nSoma de todos os quadrados: {gr.sum()}")

gini_clara = 1 - gr.sum()

print(f"Gini Index da cerveja clara {gini_clara}")

Tamanho do DF Clara: 6

Quantidade de elementos em cada classe:
Aceita
Não    2
Sim    4
Name: Cor, dtype: int64

% de elementos em cada classe:
Aceita
Não    0.333333
Sim    0.666667
Name: Cor, dtype: float64

% de elementos ao quadrado em cada classe:
Aceita
Não    0.111111
Sim    0.444444
Name: Cor, dtype: float64

Soma de todos os quadrados: 0.5555555555555556
Gini Index da cerveja clara 0.4444444444444444


### Escura

In [101]:
size_escura = len(group_escura['Aceita'])
print(f"Tamanho do DF Escura: {size_escura}\n")

gr = group_escura.groupby("Aceita")["Cor"].count()
print(f"Quantidade de elementos em cada classe:")
print(gr)

gr = gr/size_escura
print(f"\n% de elementos em cada classe:")
print(gr)

gr = gr**2
print(f"\n% de elementos ao quadrado em cada classe:")
print(gr)

print(f"\nSoma de todos os quadrados: {gr.sum()}")

gini_escura = 1 - gr.sum()
print(f"Gini Index da cerveja clara {gini_escura}")

Tamanho do DF Escura: 4

Quantidade de elementos em cada classe:
Aceita
Não    3
Sim    1
Name: Cor, dtype: int64

% de elementos em cada classe:
Aceita
Não    0.75
Sim    0.25
Name: Cor, dtype: float64

% de elementos ao quadrado em cada classe:
Aceita
Não    0.5625
Sim    0.0625
Name: Cor, dtype: float64

Soma de todos os quadrados: 0.625
Gini Index da cerveja clara 0.375


## 3. Agregar os Gini indexes de cada classe com uma soma ponderada pelo tamanho da classe

In [89]:
# Soma ponderada
gini_cor = (gini_clara * size_clara + gini_escura * size_escura)/(size_clara + size_escura)
print(gini_cor)

0.41666666666666663


## Entenderam? Então vamos usar uma função pra isso

In [94]:
def gini_index(groups, classes):
    total = sum(len(group) for group in groups)
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = list(group['Aceita']).count(class_val) / size
            score += p ** 2
        gini += (1.0 - score) * (size / total)
    return gini

## Gini da Cor

In [95]:
gini_cor = gini_index([group_clara, group_escura], ['Sim', 'Não'])

print(gini_cor)

0.4166666666666667


## Gini do IBU = 40

In [96]:
gini_ibu = gini_index(
    [df[df['IBU'] <= 40], df[df['IBU'] > 40]],
    ['Sim', 'Não']
)

print(gini_ibu)

0.4761904761904763


## Gini do Alcool = 5.5

In [97]:
gini_alcool = gini_index(
    [df[df['Teor_Alcool'] <= 5.5], df[df['Teor_Alcool'] > 5.5]],
    ['Sim', 'Não']
)

print(gini_alcool)

0.48


No algoritmo de Árvores de Decisão a separação sempre começa com o menor Gini (menor impureza), no nosso exemplo a Cor é a melhor separação.

## Fique Conectado

- [![YouTube](https://img.icons8.com/?size=40&id=19318&format=png&color=000000)](https://www.youtube.com/@LigaDataScience/videos)  
  Explore nossos vídeos educacionais e webinars sobre ciência de dados, machine learning e inteligência artificial. Inscreva-se para não perder nenhuma atualização!

- [![LinkedIn](https://img.icons8.com/?size=40&id=13930&format=png&color=000000)](https://www.linkedin.com/company/liga-data-science-ufpr/)  
  Siga-nos no LinkedIn para as últimas novidades, oportunidades de carreira e networking profissional no campo da ciência de dados.

- [![Instagram](https://img.icons8.com/?size=40&id=32323&format=png&color=000000)](https://www.instagram.com/ligadatascience/)  
  Confira nosso Instagram para conteúdos dos bastidores, destaques de eventos e o dia a dia da Liga Data Science. Faça parte da nossa jornada!
  
  ## Autores

<a href="https://www.linkedin.com/in/eduardopecora/" target="_blank">Eduardo Pecora</a>

## Log de modificações

| Data | Versão | Modificado por | Descrição |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 29-05-2025       | 1.0     | Eduardo Pecora | Inicial               |

<hr>

## <h3 align="center"> (c) Liga Data Science/ UFPR 2025. All rights reserved. <h3/>